In [ ]:
pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.4 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0


In [ ]:
#%%writefile app.py
import streamlit as st
import numpy as np
import pandas as pd
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.svm import SVC
from sklearn.impute import SimpleImputer

#StackingClassifier için base model ve meta model tanımlamaları

#  StackingClassifier, birden fazla temel modelin (base model) tahminlerini bir araya getirerek daha iyi bir tahmin yapmaya çalışan bir sınıflandırıcıdır.
# Bu model, meta-model olarak adlandırılan bir modeli kullanarak temel modellerin tahminlerini birleştirir.

#  Base Models (Temel Modeller): Bu modeller, farklı algoritmaları veya farklı hiperparametre ayarlarını kullanarak oluşturulan çeşitli
# sınıflandırıcı veya regresyon modelleridir. Örneğin, RandomForestClassifier, LogisticRegression, SVC gibi farklı algoritmaları veya bu algoritmaların farklı
# hiperparametre kombinasyonlarını temsil edebilirler.

#  Meta Model: Bu model, temel modellerin tahminlerini bir araya getirerek son tahmini oluşturur.
# Tipik olarak, meta-model daha basit bir modeldir ve temel modellerin çıktılarını birleştirmek veya ağırlamak için kullanılır.
# Örneğin, meta-model olarak genellikle Logistic Regression veya RandomForest gibi daha basit bir model kullanılır.

base_models = [
    ('rf', RandomForestClassifier(n_estimators=10, random_state=42)),
    ('svm', SVC())
]

meta_model = RandomForestClassifier(n_estimators=100, random_state=42)

#İnternet sitesinin sidebarına radio butonlar ekler. Sayfalar arası gezmek için kullanılmıştır.
rad= st.sidebar.radio("Menu",["Home","Mantar Yenilebilirliği","Su İçilebilirliği"])

#Anasayfa
if rad=="Home":
  st.title("201505032 Kerim Berkay Buçan")
  st.text("Aşağıdaki datasetler üzerine karar destek sistemi oluşturulacaktır")
  st.text("1. Mantarın Yenilebilirliğinin Tespiti")
  st.text("2. Suyun İçilebilirliğinin Tespiti ")



Overwriting app.py


In [ ]:
#%%writefile -a app.py
#Mantar verilerinin okunması ve verilerin oluşturulması
#Mantar verileri string ifade olarak verildiği için OneHotEncoder ile binary sayılara çevrilmiştir.

data_mushroom=pd.read_csv("/content/mushrooms.csv")


# Bu sütunlar, mantarların özelliklerini tanımlayan özniteliklerdir.
categorical_columns_mushroom = ['cap-shape', 'cap-surface', 'cap-color', 'bruises', 'odor',
                       'gill-attachment', 'gill-spacing', 'gill-size', 'gill-color',
                       'stalk-shape', 'stalk-root', 'stalk-surface-above-ring',
                       'stalk-surface-below-ring', 'stalk-color-above-ring', 'stalk-color-below-ring',
                       'veil-type', 'veil-color', 'ring-number', 'ring-type',
                       'spore-print-color', 'population', 'habitat']

# OneHotEncoder sınıfından bir nesne oluşturur. Bu, kategorik sütunları ikili (binary) vektörler haline getirerek
#makine öğrenimi modellerine uygun hale getirmek için kullanılır. 'drop="first"' parametresi, ilk kategoriyi düşürerek çoklu doğrusallığı önler.
encoder_mushroom = OneHotEncoder(drop='first', sparse=False)

#OneHotEncoder'ı kullanarak kategorik sütunları dönüştürür
encoded_data_mushroom = encoder_mushroom.fit_transform(data_mushroom[categorical_columns_mushroom])
#Encode edilmiş veriyi bir DataFrame'e dönüştürür.
encoded_df_mushroom = pd.DataFrame(encoded_data_mushroom, columns=encoder_mushroom.get_feature_names_out(categorical_columns_mushroom))

# Bağımsız değişkenleri (X) ve bağımlı değişkeni (y) belirler. X, encode edilmiş DataFrame'i temsil ederken,
# y "class" sütununu temsil eder, yani mantarın zehirli mi yoksa yenilebilir mi olduğunu gösterir.
x_mushroom = encoded_df_mushroom
y_mushroom = data_mushroom['class']

# Veri kümesini eğitim ve test setlerine ayırır.
# random_state=42  parametresi, veri setini rastgele bölerken tekrarlanabilirliği sağlar.
xmushroom_train, xmushroom_test, ymushroom_train, ymushroom_test = train_test_split(x_mushroom, y_mushroom, test_size=0.2, random_state=42)

#Random Forest modelinin oluşturulup eğitilmesi
mushRoomRf=RandomForestClassifier()
mushRoomRf.fit(xmushroom_train,ymushroom_train)

# #Adaboost modelinin oluşturulup eğitilmesi
mushRoomAb=AdaBoostClassifier()
mushRoomAb.fit(xmushroom_train,ymushroom_train)

# #BaggingClassifier modelinin oluşturulup eğitilmesi
mushRoomBg=BaggingClassifier()
mushRoomBg.fit(xmushroom_train,ymushroom_train)

# #StackingClassifier modelinin oluşturulup eğitilmesi
mushRoomSg=StackingClassifier(estimators=base_models, final_estimator=meta_model)
mushRoomSg.fit(xmushroom_train,ymushroom_train)







Appending to app.py


In [ ]:
#%%writefile -a app.py
#Mushroom Tahmin
if rad=="Mantar Yenilebilirliği":
  st.header("Mantar Yenilebilir Mi Yenilemez Mi")
  #Tahmin yapılması için kullanıcıdan selectboxlarla verilerin alınması
  cap_shape = st.selectbox(
    'Kapak Şekli? çan = B, konik = C, dışbükey = X, düz = F, topuzlu = K, batık = S',
    ('b', 'c', 'x', 'f', 'k', 's'))
  cap_surface = st.selectbox(
    'Kapak Yüzeyi? lifli = F, oluklar = G, pullu = Y, pürüzsüz = s',
    ('f', 'g', 'y', 's'))
  cap_color = st.selectbox(
    'Kapak Rengi? Kahverengi = N, Buff = B, Tarçın = C, Gri = G, Yeşil = R, Pembe = P, Mor = U, Kırmızı = E, Beyaz = W, Sarı = Y',
    ('n', 'b', 'c', 'g', 'r', 'p', 'u', 'e', 'w', 'y'))
  bruises = st.selectbox(
    'Çürük? Evet = t, Hayır = f',
    ('t', 'f'))
  odor = st.selectbox(
    'Koku? badem = A, Anason = L, Kreozot = C, Balık = Y, faul = F, küflü = M, Yok = N, Keskin = P, baharatlı = S',
    ('a', 'l', 'c', 'y', 'f', 'm', 'n', 'p', 's'))
  gill_attachment = st.selectbox(
    'Solungaç eki? ekli = A, azalan = D, serbest = F, çentikli = n',
    ('a', 'd', 'f', 'n'))
  gill_spacing = st.selectbox(
    'solungaç aralığı? yakın = c, kalabalık = w, uzak = d',
    ('c', 'w', 'd'))
  gill_size = st.selectbox(
    'solungaç boyutu? geniş = b, dar = n',
    ('b', 'n'))
  gill_color= st.selectbox(
    'Solungaç rengi: siyah = k, kahverengi = n, buff = b, çikolata = h, gri = g, yeşil = r, turuncu = o, pembe = p, mor = u, kırmızı = e, beyaz = w, sarı = y',
    ('k', 'n','b','h','g','r','o''p','u','e','w','y'))
  stalk_shape = st.selectbox(
    'Sap şekli?  büyütme = E, sivriltme = t',
    ('e', 't'))

  stalk_root = st.selectbox(
    'Sap kökü? soğanlı = b, kulüp = c, fincan = u, eşit = e, rizomorflar = z, köklü = r, eksik =?',
    ('b', 'c', 'u', 'e', 'z', 'r','?'))


  stalk_surface_above_ring = st.selectbox(
    'Halkanın üstünde sap yüzeyi? lifli = F, pullu = y, ipeksi = k, pürüzsüz = s',
    ('f', 'y', 'k', 's'))
  stalk_surface_below_ring = st.selectbox(
    'Halkanın altında sap yüzeyi? lifli = F, pullu = y, ipeksi = k, pürüzsüz = s',
    ('f', 'y', 'k', 's'))


  stalk_color_above_ring = st.selectbox(
    'Halkanın Üstünde Sap Rengi? Kahverengi = N, Buff = B, Tarçın = C, Gri = G, Turuncu = O, Pembe = P, Kırmızı = E, Beyaz = W, Sarı = Y',
    ('n', 'b', 'c', 'g', 'o', 'p', 'e', 'w', 'y'))


  stalk_color_below_ring = st.selectbox(
    'Halkanın altındaki sap rengi? kahverengi = n, buff = b, tarçın = c, gri = g, turuncu = o, pembe = p, kırmızı = e, beyaz = w, sarı = y',
    ('n', 'b', 'c', 'g', 'o', 'p', 'e', 'w', 'y'))


  veil_type = st.selectbox(
    'peçe tipi? kısmi=p,evrensel=u',
    ('p', 'u'))

  veil_color = st.selectbox(
    'peçe rengi: kahverengi = n, turuncu = o, beyaz = w, sarı = y',
    ('n', 'o', 'w', 'y'))
  ring_number = st.selectbox(
    'Zil numarası: Yok = N, Bir = O, İki = T',
    ('n', 'o', 't'))
  ring_type = st.selectbox(
    'Halka tipi: örümcek ağı = C, Evanescent = E, Parlama = F, Büyük = L, Yok = N, Kolye = P, Kılıf = S, Bölge = Z',
    ('e', 'f', 'l', 'n', 'p'))
  spore_print_color = st.selectbox(
    'spor-baskı-renk: siyah = k, kahverengi = n, buff = b, çikolata = h, yeşil = r, turuncu = o, mor = u, beyaz = w, sarı = y',
    ('k', 'n', 'b', 'h', 'r', 'o', 'u', 'w', 'y'))
  population = st.selectbox(
    'Nüfus: Bol=A,Kümeli=C,Sayılı=N,Dağınık=S,Birkaç=V,Yalnız=Y',
    ('a', 'c', 'n', 's', 'v', 'y'))
  habitat = st.selectbox(
    'Habitat: otlar = g, yapraklar = l, çayırlar = m, yollar = p, kentsel = u, atık = w, ormanlar = d',
    ('g', 'l', 'm', 'p', 'u', 'w', 'd'))

  #Veriler string değerler olarak alındığı için encode edilmesi için dataframe oluşturulması.
  user_input_mushroom = pd.DataFrame({
    'cap-shape': [cap_shape],
    'cap-surface': [cap_surface],
    'cap-color': [cap_color],
    'bruises': [bruises],
    'odor': [odor],
    'gill-attachment': [gill_attachment],
    'gill-spacing': [gill_spacing],
    'gill-size': [gill_size],
    'gill-color': [gill_color],
    'stalk-shape': [stalk_shape],
    'stalk-root': [stalk_root],
    'stalk-surface-above-ring': [stalk_surface_above_ring],
    'stalk-surface-below-ring': [stalk_surface_below_ring],
    'stalk-color-above-ring': [stalk_color_above_ring],
    'stalk-color-below-ring': [stalk_color_below_ring],
    'veil-type': [veil_type],
    'veil-color': [veil_color],
    'ring-number': [ring_number],
    'ring-type': [ring_type],
    'spore-print-color': [spore_print_color],
    'population': [population],
    'habitat': [habitat]
    })

  user_encoded = encoder_mushroom.transform(user_input_mushroom)

  user_encoded_df = pd.DataFrame(user_encoded, columns=encoder_mushroom.get_feature_names_out(categorical_columns_mushroom))
  tahminMushRoomRf=mushRoomRf.predict(user_encoded_df)
  tahminMushRoomAb=mushRoomAb.predict(user_encoded_df)
  tahminMushRoomBg=mushRoomBg.predict(user_encoded_df)
  tahminMushRoomSg=mushRoomSg.predict(user_encoded_df)


  if st.button("Mushroom Predict"):
    st.text("Random Forest")
    if tahminMushRoomRf[0]=="e":
      st.success("Bu mantar yenilebilir. Afiyet Olsun")
    elif tahminMushRoomRf[0]=="p":
      st.warning("Bu mantar zehirlidir. Eğer tükettiyseniz en yakın sağlık kuruluşuna gidiniz.")

    st.text("Ada Boost")
    if tahminMushRoomAb=="e":
      st.success("Bu mantar yenilebilir. Afiyet Olsun")
    elif tahminMushRoomAb=="p":
      st.warning("Bu mantar zehirlidir. Eğer tükettiyseniz en yakın sağlık kuruluşuna gidiniz.")
    st.text("Bagging")
    if tahminMushRoomBg=="e":
      st.success("Bu mantar yenilebilir. Afiyet Olsun")
    elif tahminMushRoomBg=="p":
      st.warning("Bu mantar zehirlidir. Eğer tükettiyseniz en yakın sağlık kuruluşuna gidiniz.")
    st.text("Stacking")
    if tahminMushRoomSg=="e":
      st.success("Bu mantar yenilebilir. Afiyet Olsun")
    elif tahminMushRoomSg=="p":
      st.warning("Bu mantar zehirlidir. Eğer tükettiyseniz en yakın sağlık kuruluşuna gidiniz.")



Appending to app.py


In [ ]:
#%%writefile -a app.py
# Su İçilebilirlik Modelinin Oluşturulması
df_water=pd.read_csv("/content/water_potability.csv")

x_water=df_water.drop("Potability",axis=1)
y_water=df_water["Potability"]
#Verinin içinde eksik değerler olduğu için ortalama stratejisiyle eksik değerlerin doldurulması (imputer kütüphanesi)
xwater_train,xwater_test,ywater_train,ywater_test=train_test_split(x_water,y_water,test_size=0.2)
imputer = SimpleImputer(strategy='mean')

xwater_train = imputer.fit_transform(xwater_train)
xwater_test = imputer.transform(xwater_test)

#Random Forest modelinin oluşturulup eğitilmesi
waterRf=RandomForestClassifier()
waterRf.fit(xwater_train,ywater_train)

# #Adaboost modelinin oluşturulup eğitilmesi
waterAb=AdaBoostClassifier()
waterAb.fit(xwater_train,ywater_train)

# #BaggingClassifier modelinin oluşturulup eğitilmesi
waterBg=BaggingClassifier()
waterBg.fit(xwater_train,ywater_train)

# #StackingClassifier modelinin oluşturulup eğitilmesi
waterSg=StackingClassifier(estimators=base_models, final_estimator=meta_model)
waterSg.fit(xwater_train,ywater_train)





Appending to app.py


In [ ]:
#%%writefile -a app.py

#Su İçilebilirlik Tahmin
if rad=="Su İçilebilirliği":
  st.header("Su İçilebilir Mi İçilemez Mi ?")

  # Kullanıcıdan tahmin için verilerin number imputla verilerin alınması
  ph = st.number_input("ph (0.0-14.0) ", min_value=0.0, max_value=14.0)
  hardness = st.number_input("Hardness (47.4-323.0)", min_value=47.4, max_value=323.0)
  solids = st.number_input("Solids (321.0-61000.2)", min_value=321.0, max_value=61000.2)
  chloramines = st.number_input("Chloramines (0.35-13.1)", min_value=0.35, max_value=13.1)
  sulfate = st.number_input("Sulfate (129.0-481.0)", min_value=129.0, max_value=481.0)
  conductivity = st.number_input("Conductivity (181.0-753.0)", min_value=181.0, max_value=753.0)
  organic_carbon = st.number_input("Organic_carbon (2.2-28.3)", min_value=2.2, max_value=28.3)
  trihalomethanes = st.number_input("Trihalomethanes (0.74-124.0)", min_value=0.74, max_value=124.0)
  turbidity = st.number_input("Turbidity (1.45-6.74)", min_value=1.45, max_value=6.74)

  #Kullanıcdan alınan verilerle modellerin eğitilmesi
  waterRfTahmin=waterRf.predict([[ph,hardness,solids,chloramines,sulfate,conductivity,organic_carbon,trihalomethanes,turbidity]])[0]
  waterAbTahmin=waterAb.predict([[ph,hardness,solids,chloramines,sulfate,conductivity,organic_carbon,trihalomethanes,turbidity]])[0]
  waterBgTahmin=waterBg.predict([[ph,hardness,solids,chloramines,sulfate,conductivity,organic_carbon,trihalomethanes,turbidity]])[0]
  waterSgTahmin=waterSg.predict([[ph,hardness,solids,chloramines,sulfate,conductivity,organic_carbon,trihalomethanes,turbidity]])[0]


  if st.button("Water Predict"):

    st.text("Random Forest")
    if waterRfTahmin==1:
      st.success("Bu su içilebilir. Afiyet olsun.")
    elif waterRfTahmin==0:
      st.warning("Bu su içilemez. Lütfen içtiyseniz en yakın sağlık kuruluşuna gidiniz.")
    st.text("Ada Boost")
    if waterAbTahmin==1:
      st.success("Bu su içilebilir. Afiyet olsun.")
    elif waterAbTahmin==0:
      st.warning("Bu su içilemez. Lütfen içtiyseniz en yakın sağlık kuruluşuna gidiniz.")
    st.text("Bagging")
    if waterBgTahmin==1:
      st.success("Bu su içilebilir. Afiyet olsun.")
    elif waterBgTahmin==0:
      st.warning("Bu su içilemez. Lütfen içtiyseniz en yakın sağlık kuruluşuna gidiniz.")
    st.text("Stacking")
    if waterSgTahmin==1:
      st.success("Bu su içilebilir. Afiyet olsun.")
    elif waterSgTahmin==0:
      st.warning("Bu su içilemez. Lütfen içtiyseniz en yakın sağlık kuruluşuna gidiniz.")



Appending to app.py


In [ ]:
!wget -q -O - ipv4.icanhazip.com

34.173.2.102


In [ ]:
!streamlit run app.py & npx localtunnel --port 8501





  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://34.173.2.102:8501

npx: installed 22 in 4.894s
your url is: https://clean-showers-cut.loca.lt
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default